# Qu'est-ce qu'il y a dans ce notebook ?

Il y a toutes les fonctions permettant l'analyse des données obtenues après le POS_Taaging

1. Calcul des fréquences d'une liste de mots
2. Récupérer les différentes formes d'un lemme

In [38]:
import json
from collections import Counter
import collections
import pandas as pd
from ipynb.fs.defs.POS_Tagging import serialisation_data
import re

## Ouverture des fichiers Json

In [2]:
def open_file(file):
    with open(file) as json_data:
        data = json.load(json_data)
    return data
#ata_str = json.dumps(data_dict)

In [3]:
nouns = open_file("nouns.json")

In [4]:
def extract_nouns(tuple_list):
    """extract different forms from tuple
    -> tuple list
    <- noun list"""
    
    form_list = []
    
    for tup in tuple_list:
        form_list.append(tup[0])
        
    return form_list

In [6]:
nouns_list = extract_nouns(nouns)

In [7]:
print(nouns_list[0:50])

['confiance', 'société', 'élus', 'rôle', 'associations', 'organisations', 'oui', 'type', 'associations', 'organisations', 'rôle', 'lien', 'citoyens', 'élus', 'non-cumul', 'mandats', 'parlementaires', 'députés', 'sensibilités', 'nombre', 'élus', 'députés', 'oui', 'participation', 'citoyens', 'élections', 'compte', 'vote', 'oui', 'manière', 'citoyens', 'orientations', 'décision', 'place', 'démocratie', 'participative', 'déclenchement', 'référendum', 'initiative', 'initiative', 'membres', 'Parlement', 'partie', 'corps', 'oui', 'citoyens', 'utilisation', 'argent', 'Etat', 'collectivités']


## Calcul des fréquences des noms

### Calcul des occurences de chaque mot

Format Dictionnaire (objet Counter)

In [8]:
def dict_occ (words_list) :
    """Count occurencies of words
    -> list of words
    <- dictionary of word frequencies like {word_1 : occurence, word_2 : occurence, ... , word_n : occurence}"""

    occ = collections.Counter(words_list)
    
    return occ

In [11]:
occ = dict_occ(nouns_list)

print("Nombre d‘occurences uniques :", len(occ))

Nombre d‘occurences uniques : 132725


In [36]:
ordered_occ_dict = sorted(occ.items(), key=lambda x:x[1], reverse = True)

print(ordered_occ_dict[0:50])
sort_dict = dict(ordered_occ_dict)
print(len(ordered_occ_dict))

[('citoyens', 181332), ('élus', 155028), ('vote', 131477), ('pays', 129567), ('chose', 102982), ('respect', 95574), ('Citoyen', 91105), ('personnes', 90165), ('vie', 85028), ('travail', 81378), ('l’', 78069), ('exemple', 75756), ('éducation', 72783), ('nombre', 67754), ('associations', 65427), ('place', 64475), ('école', 62695), ('société', 62044), ('temps', 60616), ('loi', 58543), ('compte', 57939), ('lois', 56294), ('gens', 53818), ('niveau', 53404), ('députés', 53388), ('%', 50739), ('politique', 49556), ('immigration', 49301), ('cas', 49236), ('démocratie', 46442), ('citoyen', 46082), ('rôle', 45388), ('proportionnelle', 45303), ('valeurs', 45242), ('élections', 43587), ('enfants', 42327), ('droit', 40788), ('état', 40549), ('service', 39462), ('intérêt', 39393), ('monde', 38243), ('mandat', 37200), ('ans', 37099), ('élection', 35675), ('question', 35557), ('élu', 35018), ('peuple', 32456), ('votes', 32169), ('pouvoir', 31109), ('décisions', 30846)]
132725


### Calcul des fréquences et tri par fréquence les plus élevées

Stockée sous forme de dictionnaire

In [13]:
def dict_freq (dict_occurence):
    """calculates frequencies of words
    <- dictionnary of occurences
    -> dictionary of frequencies like {word_1 : frequence, word_2 : frequence, ... , word_n : frequence}"""

    data_fq = {key : dict_occurence[key]/len(dict_occurence) for key in dict_occurence}
    
    return data_fq


In [14]:
freq = dict_freq(occ)

#### Tri du dictionnaire par fréquence les plus élevées

In [28]:
ordered_freq_dict = sorted(freq.items(), key=lambda x:x[1], reverse = True)

print(ordered_freq_dict[0:50])
sortdict = dict(ordered_freq_dict)

[('citoyens', 1.3662233942362028), ('élus', 1.1680391787530608), ('vote', 0.9905970992653984), ('pays', 0.9762064418911283), ('chose', 0.7759050668675833), ('respect', 0.7200904125070635), ('Citoyen', 0.6864192880015069), ('personnes', 0.6793369749482012), ('vie', 0.6406328875494444), ('travail', 0.6131324166509701), ('l’', 0.5882011678282162), ('exemple', 0.5707741570917311), ('éducation', 0.5483744584667546), ('nombre', 0.510484083631569), ('associations', 0.4929515916368431), ('place', 0.4857788660764739), ('école', 0.4723676775287248), ('société', 0.4674627990205312), ('temps', 0.4567037106799774), ('loi', 0.44108495008476173), ('compte', 0.4365341872292334), ('lois', 0.4241401393859484), ('gens', 0.4054850254285176), ('niveau', 0.40236579393482763), ('députés', 0.40224524392540967), ('%', 0.3822866829911471), ('politique', 0.373373516669806), ('immigration', 0.3714522508947071), ('cas', 0.3709625164814466), ('démocratie', 0.3499114710868337), ('citoyen', 0.34719909587492936), ('rô

## Récupérer les différentes formes d'un mot

In [15]:
def get_lemma_form (tuple_list):
    """creates a dictionnary of different forms for each lemma of the processed document
    -> liste of tuples like [(word_form1, word_lemma1), ..., (word_formn, word_lemman)
    <- dictionnary with as key a lemma and as values the different forms found in the text
    dictionary form : {lemma1 : [form1,form2, formn], lemma2 : [form1,form2,formn], ..., lemman: [forma1,form2,formn]}
    """
    
    lemma_and_form = {}
    
    for tup in tuple_list:
        if tup[1] not in lemma_and_form: lemma_and_form[tup[1]] = []
        if tup[0] not in lemma_and_form[tup[1]]: lemma_and_form[tup[1]].append(tup[0])
        
    return lemma_and_form

In [16]:
lemma_form = get_lemma_form(nouns)

In [60]:
print("Nombre de lemme différents : ", len(lemma_form))

Nombre de lemme différents :  249


### Sérialisation des données

Via la fonction serialisation_data de POS_tagging.ipynb

In [17]:
def serialisation_data (data, title):
  """
  Serialize data in a json file
  -> Title mus be a string : title.json
  <- Save a file in desktop
  """

  with open(title, "w+") as file:
    json.dump(data, file)

In [30]:
#Sérialisation du dictionnaire d'occurence
#serialisation_data (occ, "dict_occ.json")

#Sérialisation du dictionnaire de fréquence
#serialisation_data (freq, "dict_freq.json")

#Sérialisation du dictionnaire de fréquence trié
serialisation_data (sortdict, "dict_ord_freq.json")

#Sérialisation du dictionnaire de lemmes
#serialisation_data (lemma_form, "lemme_form.json")

In [ ]:
data_frame = pd.DataFrame.from_dict(lemma_form, orient='index')

print(data_frame)
col = data_frame.shape[1]

print(col)

data_frame.to_csv(r"./ lemmes.csv", header = [i for i in range(1,col+1)])

In [25]:
data_frame = pd.DataFrame.from_dict(freq, orient='index')

print(data_frame)
col = data_frame.shape[1]

print(col)

data_frame.to_csv(r"./ fréquences.csv", header = ["Fréquence"])

                                 0
confiance                 0.221722
société                   0.467463
élus                      1.168039
rôle                      0.341970
associations              0.492952
...                            ...
hyperpresidentialisation  0.000008
efficae                   0.000008
luie                      0.000008
Contournement             0.000008
QPS                       0.000008

[132725 rows x 1 columns]
1


In [32]:
data_frame = pd.DataFrame.from_dict(sortdict, orient='index')

print(data_frame)

data_frame.to_csv(r"./ fréquences_ordered.csv", header = ["Fréquence"])

                                 0
citoyens                  1.366223
élus                      1.168039
vote                      0.990597
pays                      0.976206
chose                     0.775905
...                            ...
hyperpresidentialisation  0.000008
efficae                   0.000008
luie                      0.000008
Contournement             0.000008
QPS                       0.000008

[132725 rows x 1 columns]


In [37]:
data_frame = pd.DataFrame.from_dict(sort_dict, orient='index')

print(data_frame)

data_frame.to_csv(r"./ occurences_ordered.csv", header = ["Fréquence"])

                               0
citoyens                  181332
élus                      155028
vote                      131477
pays                      129567
chose                     102982
...                          ...
hyperpresidentialisation       1
efficae                        1
luie                           1
Contournement                  1
QPS                            1

[132725 rows x 1 columns]
